<a href="https://colab.research.google.com/github/aliakbarbadri/persian-poetry-creator/blob/master/char-RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [6]:
! wget --no-check-certificate https://github.com/aliakbarbadri/persian-poetry-creator/blob/master/Shahnameh.zip?raw=true -O /tmp/Shahnameh.zip
! unzip /tmp/Shahnameh.zip 

--2020-05-26 14:33:37--  https://github.com/aliakbarbadri/persian-poetry-creator/blob/master/Shahnameh.zip?raw=true
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/aliakbarbadri/persian-poetry-creator/raw/master/Shahnameh.zip [following]
--2020-05-26 14:33:38--  https://github.com/aliakbarbadri/persian-poetry-creator/raw/master/Shahnameh.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aliakbarbadri/persian-poetry-creator/master/Shahnameh.zip [following]
--2020-05-26 14:33:38--  https://raw.githubusercontent.com/aliakbarbadri/persian-poetry-creator/master/Shahnameh.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercon

In [0]:
import os
corpus_url = "Shahnameh/shahname"
corpus = ""
for filename in os.listdir(corpus_url):
   with open(os.path.join(corpus_url, filename), 'r') as f:
      text = f.read()
      corpus += text
      corpus += "\n"

In [0]:
file_corpus = open("shahname.txt", "a")
file_corpus.write(corpus)
file_corpus.close()

In [0]:
# filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url) with open(filepath) as f:
# shakespeare_text = f.read()

In [18]:
corpus.split("\n")[0]

'یکی راه پیش آمدش ناگزیر\tهمی رفت بایست بر خیره خیر'

In [19]:
len(corpus)

2555369

In [0]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True) 
tokenizer.fit_on_texts([corpus])

In [21]:
tokenizer.texts_to_sequences(["سلام"])

[[14, 25, 2, 10]]

In [24]:
tokenizer.sequences_to_texts([[14, 25, 2, 10]])

['س ل ا م']

In [25]:
"".join(sorted(set(corpus)))

'\t\n ()«»،؟ءآأؤئابتثجحخدذرزسشصضطظعغفقلمنهؤپچژکگی\u200c'

In [26]:
max_id = len(tokenizer.word_index)
max_id

48

In [27]:
dataset_size = len(corpus)
dataset_size

2555369

In [0]:
[encoded] = np.array(tokenizer.texts_to_sequences([corpus])) - 1

In [0]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [0]:
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead 
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [0]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [0]:
np.random.seed(42)
tf.random.set_seed(42)
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [36]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 48) (32, 100)


In [45]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, steps_per_epoch=train_size // batch_size,
                    epochs=10)

Epoch 1/10
 9803/71869 [===>..........................] - ETA: 49:04 - loss: 1.1469

KeyboardInterrupt: ignored

In [0]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [0]:
X_new = preprocess(["که ایران چو باغی ست خرم بهار"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

In [0]:
tf.random.set_seed(42)

tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=40).numpy()

In [0]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [0]:
tf.random.set_seed(42)

next_char("How are yo", temperature=1)

In [0]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [0]:
tf.random.set_seed(42)

print(complete_text("t", temperature=0.2))

In [0]:
print(complete_text("t", temperature=1))

In [0]:
print(complete_text("t", temperature=2))

In [0]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True) 
dataset = dataset.flat_map(lambda window: window.batch(window_length)) 
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:])) 
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch)) 
dataset = dataset.prefetch(1)

In [49]:
model = keras.models.Sequential([
keras.layers.GRU(128, return_sequences=True, stateful=True,
dropout=0.2, recurrent_dropout=0.2,
batch_input_shape=[batch_size, None, max_id]), keras.layers.GRU(128, return_sequences=True, stateful=True,
dropout=0.2, recurrent_dropout=0.2), keras.layers.TimeDistributed(keras.layers.Dense(max_id,
activation="softmax"))
])